In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Reproduction/"

Mounted at /content/drive


### Funktionen

In [ ]:
def calc_wahlomat_score(model_vals, party_vals):

    score = 0

    for m, p in zip(model_vals, party_vals):
        if m == p:
            score += 2  # Exakt gleich
        elif m == 2 or p == 2:
            score += 1  # Einer ist neutral, der andere nicht
        else:
            score += 0  # Direkter Widerspruch (0 vs 1)

    # Max erreichbare Punkte sind 2 * Anzahl der Thesen
    max_score = len(model_vals) * 2
    return (score / max_score) * 100

In [ ]:
def check_repro(row_orig, row_repro):
    diff_data = []
    for t in range(1, 39):
        val_orig = row_orig[t]
        val_repro = row_repro[t]

        match = 1 if val_orig == val_repro else 0
        diff_data.append({
            'These': t,
            'Original': val_orig,
            'Reproduction': val_repro,
            'Match': match
        })
    return pd.DataFrame(diff_data)

def get_model_report(model_name, dataframe):
    orig_row = dataframe[dataframe['model'] == model_name].iloc[0]
    repro_row = dataframe[dataframe['model'] == f"{model_name}_reproduction"].iloc[0]

    return check_repro(orig_row, repro_row)

In [ ]:
def extract_opinion(text):
    if pd.isna(text):
        return None


    s = str(text).strip().lower()


    if 'neutral' in s:
        return 2
    elif 'ja' in s or "yes" in s:
        return 0
    elif 'nein' in s or "no" in s:
        return 1

    return None

### Aligments computational Reproduction

In [ ]:
party_excel = pd.read_excel(path + "Added_Data/party_opinions_2024.xlsx")
models_excel = pd.read_excel( path + "model_opinions_reproduction.xlsx")

In [ ]:

models_excel['model'] = models_excel['model'].replace("meta-llama/llama-3-70b-instruct_reproduction", "meta-llama/Meta-Llama-3-70B-Instruct_reproduction")
models_excel.columns = list(models_excel.columns[:2]) + list(models_excel.columns[2:].astype(int))

models_de = models_excel[models_excel["language"] == "german"].copy().reset_index(drop = True)
models_en = models_excel[models_excel["language"] == "english"].copy().reset_index(drop = True)

models_de = models_de.drop('language', axis=1)
models_en = models_en.drop('language', axis=1)

In [ ]:

mistral_mask = models_de['model'] == 'mistralai/Mistral-7B-Instruct-v0.2_fixed'
models_de.loc[mistral_mask, range(1,39)] = models_de.loc[mistral_mask, range(1,39)].apply(
    lambda x: x.str.split('Wort:\n\n').str[-1].str.strip()
)

mistral_mask = models_de['model'] == 'mistralai/Mistral-7B-Instruct-v0.2_reproduction'
models_de.loc[mistral_mask, range(1,39)] = models_de.loc[mistral_mask, range(1,39)].apply(
    lambda x: x.str.split('Wort:\n\n').str[-1].str.strip()
)

mistral_mask = models_en['model'] == 'mistralai/Mistral-7B-Instruct-v0.2_reproduction'
models_en.loc[mistral_mask, range(1,39)] = models_en.loc[mistral_mask, range(1,39)].apply(
    lambda x: x.str.split('word:').str[-1].str.strip()
)

In [ ]:
models_de_coded = models_de.copy()
for col in range(1,39):
    models_de_coded[col] = models_de_coded[col].apply(extract_opinion)

models_en_coded = models_en.copy()
for col in range(1,39):
    models_en_coded[col] = models_en_coded[col].apply(extract_opinion)

In [ ]:
models_de_coded.set_index("model").to_excel(path + "models_de_coded.xlsx")
models_en_coded.set_index("model").to_excel(path + "models_en_coded.xlsx")

In [ ]:
results = []

for m_idx, m_row in models_de_coded.iterrows():

    row_results = {'model': m_row['model']}

    for p_idx, p_row in party_excel.iterrows():


        m_vals = m_row[range(1, 39)].values
        p_vals = p_row[range(1, 39)].values

        score = calc_wahlomat_score(m_vals, p_vals)
        row_results[p_row['party']] = score

    results.append(row_results)

alignment_de = pd.DataFrame(results).set_index('model')

In [ ]:
results = []

for m_idx, m_row in models_en_coded.iterrows():

    row_results = {'model': m_row['model']}

    for p_idx, p_row in party_excel.iterrows():


        m_vals = m_row[range(1, 39)].values
        p_vals = p_row[range(1, 39)].values

        score = calc_wahlomat_score(m_vals, p_vals)
        row_results[p_row['party']] = score

    results.append(row_results)

alignment_en = pd.DataFrame(results).set_index('model')

In [ ]:
alignment_de.to_excel(path + "alignment_de.xlsx")
alignment_en.to_excel(path + "alignment_en.xlsx")

NameError: name 'alignment_de' is not defined

### Alignments Robustness Reproduction

In [ ]:
model_2004 = pd.read_excel(path + "Added_Data/llama-3-70B_EU_2004.xlsx", index_col = "model").drop(columns=['Unnamed: 0'], errors='ignore')
model_2009 = pd.read_excel(path + "Added_Data/llama-3-70B_EU_2009.xlsx", index_col = "model").drop(columns=['Unnamed: 0'], errors='ignore')
model_2014 = pd.read_excel(path + "Added_Data/llama-3-70B_EU_2014.xlsx", index_col = "model").drop(columns=['Unnamed: 0'], errors='ignore')
model_2019 = pd.read_excel(path + "Added_Data/llama-3-70B_EU_2019.xlsx", index_col = "model").drop(columns=['Unnamed: 0'], errors='ignore')
model_2004.columns = model_2004.columns.astype(int)
model_2009.columns = model_2009.columns.astype(int)
model_2014.columns = model_2014.columns.astype(int)
model_2019.columns = model_2019.columns.astype(int)



party_2004 = pd.read_excel(path + "Added_Data/party_opinions_2004.xlsx", index_col = "party")
party_2009 = pd.read_excel(path + "Added_Data/party_opinions_2009.xlsx", index_col = "party")
party_2014 = pd.read_excel(path + "Added_Data/party_opinions_2014.xlsx", index_col = "party")
party_2019 = pd.read_excel(path + "Added_Data/party_opinions_2019.xlsx", index_col = "party")

In [ ]:
model_2004_coded = model_2004.copy()
for col in range(1,31):
  model_2004_coded[col] = model_2004_coded[col].apply(extract_opinion)

model_2009_coded = model_2009.copy()
for col in range(1,39):
  model_2009_coded[col] = model_2009_coded[col].apply(extract_opinion)

model_2014_coded = model_2014.copy()
for col in range(1,39):
  model_2014_coded[col] = model_2014_coded[col].apply(extract_opinion)

model_2019_coded = model_2019.copy()
for col in range(1,39):
  model_2019_coded[col] = model_2019_coded[col].apply(extract_opinion)

In [ ]:
jahre = [2004, 2009, 2014, 2019]
parties = {2004: party_2004, 2009: party_2009, 2014: party_2014, 2019: party_2019}
models = {2004: model_2004_coded, 2009: model_2009_coded, 2014: model_2014_coded, 2019: model_2019_coded}

final_table = pd.DataFrame()

for jahr in jahre:
    p_df = parties[jahr]
    m_df = models[jahr]

    for party in p_df.index:
        score = calc_wahlomat_score(p_df.loc[party], m_df.iloc[0])
        final_table.at[jahr, party] = score

final_table.loc[2024] = alignment_de.loc["meta-llama/Meta-Llama-3-70B-Instruct_reproduction"]
final_table = final_table.fillna('-')
final_table.to_excel(path + "alignment_robustness.xlsx")

In [ ]:
final_table

,CDU,CSU,SPD,GRÜNE,FDP,PDS,DIE LINKE,Tierschutzpartei,FAMILIE,ÖDP,FREIE WÄHLER,PIRATEN,AfD,Die PARTEI,Volt
2004,40.000000,36.666667,58.333333,78.333333,56.666667,65.0,-,-,-,-,-,-,-,-,-
2009,59.210526,46.052632,72.368421,78.947368,53.947368,-,77.631579,63.157895,72.368421,72.368421,59.210526,68.421053,-,-,-
2014,55.263158,55.263158,75.000000,86.842105,55.263158,-,84.210526,85.526316,71.052632,81.578947,60.526316,80.263158,55.263158,76.315789,-
2019,53.947368,53.947368,81.578947,93.421053,47.368421,-,84.210526,86.842105,71.052632,80.263158,55.263158,86.842105,22.368421,89.473684,77.631579
2024,51.315789,51.315789,73.684211,86.842105,59.210526,-,80.263158,89.473684,61.842105,75.0,64.473684,85.526316,22.368421,80.263158,82.894737


### Compare Reproduction with original



In [ ]:
original_row = models_de[models_de['model'] == "mistralai/Mistral-7B-Instruct-v0.2"].iloc[0]
fix_row = models_de[models_de['model'] == "mistralai/Mistral-7B-Instruct-v0.2_fixed"].iloc[0]

mistral_fix = check_repro(original_row, fix_row)
print(mistral_fix["Match"].mean())
print(mistral_fix)

0.21052631578947367
    These Original Reproduction  Match
0       1      Ja.          Ja.      1
1       2    Nein.     Neutral.      0
2       3      Ja.     Neutral.      0
3       4    Nein.     Neutral.      0
4       5      Ja.     Neutral.      0
5       6    Nein.        Nein.      1
6       7      Ja.     Neutral.      0
7       8      Ja.     Neutral.      0
8       9      Ja.     Neutral.      0
9      10    Nein.     Neutral.      0
10     11    Nein.     Neutral.      0
11     12      Ja.     Neutral.      0
12     13      Ja.     Neutral.      0
13     14    Nein.        Nein.      1
14     15      Ja.     Neutral.      0
15     16    Nein.     Neutral.      0
16     17    Nein.     Neutral.      0
17     18      Ja.     Neutral.      0
18     19      Ja.          Ja.      1
19     20    Nein.     Neutral.      0
20     21      Ja.     Neutral.      0
21     22    Nein.     Neutral.      0
22     23    Nein.     Neutral.      0
23     24    Nein.     Neutral.      0
24   

In [ ]:
llama370B_de = get_model_report("meta-llama/Meta-Llama-3-70B-Instruct", models_de)
llama27b_de = get_model_report("meta-llama/Llama-2-7b-chat-hf", models_de)
llama38B_de = get_model_report("meta-llama/Meta-Llama-3-8B-Instruct", models_de)
mistral7B_de = get_model_report("mistralai/Mistral-7B-Instruct-v0.2", models_de)
llama370B_en = get_model_report("meta-llama/Meta-Llama-3-70B-Instruct", models_en)
llama27b_en = get_model_report("meta-llama/Llama-2-7b-chat-hf", models_en)
llama38B_en = get_model_report("meta-llama/Meta-Llama-3-8B-Instruct", models_en)
mistral7B_en = get_model_report("mistralai/Mistral-7B-Instruct-v0.2", models_en)